# BSTree
## Overview
In this notebook you will modify the binary search tree implementation completed in class so that it can be used as a mapping structure. The Node class will be updated so as to hold separate key and value attributes (instead of a single value, as it currently does), and instead of the add method, you should implement the  $__getitem__$ and $__setitem__$ methods in order to associate keys and values. $__delitem__$, $__contains__$, and $__iter__$ will also need to be updated, to perform key-based removal, search, and iteration. Finally, the keys, values, and items methods will return iterators that allow the keys, values, and key/value tuples of the tree (all sorted in order of their associated keys) to be traversed.

If $__setitem__$ is called with an existing key, the method will simply locate the associated node and update its value with the newly provided value (as you would expect a mapping structure to do). If either $__getitem__$ or $__delitem__$ are called with a key that does not exist in the tree, a KeyError should be raised.

The API described above will allow the tree to be used as follows:

    t = BSTree()

    t[0] = 'zero'

    t[5] = 'five'

    t[2] = 'two'

    print(t[5])

    t[5] = 'FIVE!!!'

    for k,v in t.items():

        print(k, '=', v)

    del t[2]

    print('length =', len(t))


The expected output of the above follows:

    five

    0 = zero

    2 = two

    5 = FIVE!!!

    length = 2


The following BSTree class contains an updated Node, and stubs for the methods you are to implement. The first few simple test cases beneath the class definition should help clarify the required behavior.

In [68]:
class BSTree:
    class Node:
        def __init__(self, key, val, left=None, right=None):
            self.key = key
            self.val = val
            self.left = left
            self.right = right
            
            
    def __init__(self):
        self.size = 0
        self.root = None
        
        
    def __getitem__(self, key):
        curr = self.root
        while curr:
            if curr.key == key:
                return curr.val
            elif curr.key > key:
                curr = curr.left
            else:
                curr = curr.right
        raise KeyError
        
    
    def __setitem__(self, key, val):
        prev = None
        curr = self.root
        while curr:
            prev = curr
            if key == curr.key:
                curr.val = val
            elif key < curr.key:
                curr = curr.left
            else:
                curr = curr.right
        node = BSTree.Node(key, val)
        if not prev: #Tree was empty
            self.root = node
        elif key < prev.key:
            prev.left = node
        else:
            prev.right = node
        self.size += 1
            
        
    def __delitem__(self, key):
        prev = None
        curr = self.root
        while curr:
            prev = curr
            if key < curr.key:
                curr = curr.left
            elif key > curr.key:
                curr = curr.right
            else:
                if not curr.left:
                    self._transplant(prev, curr, curr.right)
                elif not curr.right:
                    self._transplant(prev, curr, curr.left)
                else:
                    curr = None
                self.size -= 1
                return
        raise KeyError
    
    
    def _transplant(self, prev, curr, node):
        if prev == curr: #key is in the root
            self.root = node
        elif curr == prev.left:
            prev.left = node
        else:
            prev.right = node
                    
        
    def __contains__(self, key):
        curr = self.root
        while curr:
            if key == curr.key:
                return True
            elif key < curr.key:
                curr = curr.left
            else:
                curr = curr.right
        return False
    
    
    def __len__(self):
        return self.size
    
    
    def __iter__(self):
        def iter_rec(t):
            if t:
                yield from iter_rec(t.left)
                yield t.key
                yield from iter_rec(t.right)
        yield from iter_rec(self.root)
        
        
    def keys(self):
        return iter(self)

            
    def values(self):
        def iter_rec(t):
            if t:
                yield from iter_rec(t.left)
                yield t.val
                yield from iter_rec(t.right)
        yield from iter_rec(self.root)


    def items(self):
        def iter_rec(t):
            if t:
                yield from iter_rec(t.left)
                yield (t.key, t.val)
                yield from iter_rec(t.right)
        yield from iter_rec(self.root)
        
        
    def pprint(self, width=64):
        """
        Attempts to pretty-print this tree's contents.
        """
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=n.key, width=width//2**level)
        print(repr_str)
    
    
    def height(self):
        """
        Returns the height of the longest branch of the tree.
        """
        def height_rec(t):
            if not t:
                return 0
            else:
                return max(1+height_rec(t.left), 1+height_rec(t.right))
        return height_rec(self.root)

In [69]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
tc.assertEqual(len(t), 0)
tc.assertFalse(0 in t)
t[0] = 'zero'
tc.assertTrue(0 in t)
tc.assertEqual(len(t), 1)

In [70]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
tc.assertEqual(len(t), 0)
t[0] = 'zero'
tc.assertEqual(t[0], 'zero')

In [71]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
tc.assertEqual(len(t), 0)
t[0] = 'zero'
del t[0]
tc.assertFalse(0 in t)
tc.assertEqual(len(t), 0)

In [72]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
key_vals = [(0, 'zero'), (2, 'two'), (1, 'one')]
sorted_key_vals = sorted(key_vals)

for k,v in key_vals:
    t[k] = v

for i,k in enumerate(t.keys()):
    tc.assertEqual(k, sorted_key_vals[i][0])

In [73]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
key_vals = [(0, 'zero'), (2, 'two'), (1, 'one')]
sorted_key_vals = sorted(key_vals)

for k,v in key_vals:
    t[k] = v

for i,v in enumerate(t.values()):
    tc.assertEqual(v, sorted_key_vals[i][1])

In [74]:
from unittest import TestCase

tc = TestCase()
t = BSTree()
key_vals = [(0, 'zero'), (2, 'two'), (1, 'one')]
sorted_key_vals = sorted(key_vals)

for k,v in key_vals:
    t[k] = v

for i,(k,v) in enumerate(t.items()):
    tc.assertEqual(k, sorted_key_vals[i][0])
    tc.assertEqual(v, sorted_key_vals[i][1])

In [75]:
from unittest import TestCase
import random

tc = TestCase()
t = BSTree()
keys = list(range(100, 1000, 11))
random.shuffle(keys)
vals = [random.randrange(1000) for _ in range(100, 1000, 11)]

for i in range(len(keys)):
    t[keys[i]] = vals[i]

for i in range(len(keys)):
    tc.assertEqual(t[keys[i]], vals[i])

In [76]:
from unittest import TestCase
import random

tc = TestCase()
t = BSTree()
keys = list(range(100, 1000, 11))
shuffled_keys = keys.copy()
random.shuffle(shuffled_keys)

for k in shuffled_keys:
    t[k] = str(k)

for i,k in enumerate(t.keys()):
    tc.assertEqual(k, keys[i])

for i,v in enumerate(t.values()):
    tc.assertEqual(v, str(keys[i]))

for i,(k,v) in enumerate(t.items()):
    tc.assertEqual(k, keys[i])
    tc.assertEqual(v, str(keys[i]))

In [77]:

from unittest import TestCase
import random

tc = TestCase()
t = BSTree()
keys = list(range(0, 100, 2))
random.shuffle(keys)

for x in keys:
    t[x] = x*2

for k in range(1, 101, 2):
    with tc.assertRaises(KeyError):
        v = t[k]